In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
import pandas as pd
import os

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
train_dir = './data/train'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2,
                                   rotation_range=5,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2)

In [ ]:
train_dataset = train_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  subset='training',
                                                  batch_size=64)

In [ ]:
valid_dataset = valid_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  batch_size=64)

https://www.kaggle.com/code/yasserhessein/emotion-recognition-with-resnet50/notebook<br>
import pretrained imagenet ResNet50 model<br>
include_top = False ->  fully-connected output layers of the model used to make predictions is not loaded<br>
allows a new output layer to be added and trained

In [ ]:
image_shape = (48, 48, 3)
num_classes = 7
epochs = 50
lr = 1e-10

In [ ]:
base_model = ResNet50(input_shape=image_shape, include_top=False, weights="imagenet")
# build model from ResNet50 base model
model = Sequential()
model.add(base_model)
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(BatchNormalization())
# # kernel_initializer = 'he_uniform' --> draws samples from a truncated normal distribution centered on 0
# model.add(Dense(32, kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(32, kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(32, kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dense(num_classes, activation='softmax'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [ ]:
lrd = ReduceLROnPlateau(monitor='val_loss', patience=20, verbose=1, factor=0.50, min_lr=lr)

In [ ]:
mcp = ModelCheckpoint('./weights/resnet50_model.h5', monitor='val_accuracy',
                      mode='max',
                      save_best_only=True)

In [ ]:
es = EarlyStopping(verbose=1, patience=20)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=valid_dataset, epochs=epochs,
                    verbose=1, callbacks=[lrd, mcp, es])

In [ ]:
os.makedirs('./history', exist_ok=True)
file_name = './history/resnet50_hist.csv'
with open(file_name, mode='w') as f:
    pd.DataFrame(history.history).to_csv(f)